## Elasticsearch with Multilingual-e5-small model

# Pre-Process

## Install necessary Packages

In [ ]:
!pip install eland elasticsearch
!pip install elasticsearch
!pip install openai==0.28
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.2/480.2 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 53.4 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.4
    Uninstalling pandas-2.1.4:
      Successfully uninstalled pandas-2.1.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.1.4, but you have pandas 1.5.3 which is incompatible.
xarray 2024.6.0 requires pandas>=2.0, but you have pandas 1.5.3 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 5.3 MB/s eta 0:00:00

## Import necessary modules

In [ ]:
from elasticsearch import (Elasticsearch,helpers)
from urllib.request import urlopen
import getpass
import json
import os

## Connect to Elasticsearch Cloud

In [ ]:
# Elastic cloud connection
CLOUD_ID = getpass.getpass("Enter Elastic Cloud ID:  ")   

# Password for the 'elastic' user generated by Elasticsearch
ELASTIC_PASSWORD = getpass.getpass("Enter Elastic password:  ")  

# Create the client instance
es = Elasticsearch(cloud_id=CLOUD_ID, basic_auth=("elastic", ELASTIC_PASSWORD), request_timeout=3600)

# Check the connection
try:
    # Ping the Elasticsearch cluster
    if es.ping():
        print("You have Successfully Connected with Elastic Cloud")
    else:
        print("Failed to connect to Elastic Cloud")
except Exception as e:
    print(f"Error connecting to Elastic Cloud: {e}")

## Create pipeline ingesting Machine Leanrning model

In [ ]:
# Creating an ingest pipeline with inference processors to use multilingual-e5-small to infer against data that will be ingested in the pipeline.


es.ingest.put_pipeline(
    id="ecommerce-pipeline_1",
    processors=[
        {
            "inference": {
                "model_id": ".multilingual-e5-small",
                "target_field": "ml",
                "field_map": {"description": "text_field"},
                "inference_config": {
                    "text_embedding": {  # text_embedding inference type
                        "results_field": "embeddings"
                    }
                },
            }
        },
    ],
)

ObjectApiResponse({'acknowledged': True})

## Create Old Index

In [ ]:
# Index to load products-ecommerce.json docs

es.indices.create(
    index="ecommerce_old_index",
    mappings={
        "properties": {
            "product": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword","ignore_above": 256}},
            },
            "description": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword","ignore_above": 256}},
            },
            "category": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword","ignore_above": 256}},
            },
        }
    },
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'ecommerce_old_index'})

## Create New Index

In [ ]:
from elasticsearch import Elasticsearch



# Reindex index
INDEX = "ecommerce_new_index"
es.indices.create(
    index=INDEX,
    settings={"index": {"number_of_shards": 1, "number_of_replicas": 1}},
    mappings={
        "properties": {
            "product": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword","ignore_above": 256}},
            },
            "description": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword","ignore_above": 256}},
            },
            "category": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword","ignore_above": 256}},
            },
        },
    },
)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'ecommerce_new_index'})

## Insert Data into Old Index

In [ ]:
#  dataset

url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/02c01b3450e8ddc72ccec85d559eee5280c185ac/supporting-blog-content/lexical-and-semantic-search-with-elasticsearch/products-ecommerce.json"  # json raw file - update the link here

response = urlopen(url)

# Load the response data into a JSON object
data_json = json.loads(response.read())


def create_index_body(doc):
    """Generate the body for an Elasticsearch document."""
    return {
        "_index": "ecommerce_old_index",
        "_source": doc,
    }


# Prepare the documents to be indexed
documents = [create_index_body(doc) for doc in data_json]

# Use helpers.bulk to index
helpers.bulk(es, documents)

print("Done indexing documents into `ecommerce_old_index` index")

Done indexing documents into `ecommerce_old_index` index


## Insert data from Old Index to New Index through Pipeline of ML model taking one targeted column embeddings

In [ ]:
# Reindex data from one index 'source' to another 'dest' with the 'ecommerce-pipeline' pipeline.

es.reindex(
    wait_for_completion=True,
    source={"index": "ecommerce_old_index"},
    dest={"index": "ecommerce_new_index", "pipeline": "ecommerce-pipeline_1"},
)

ObjectApiResponse({'took': 71333, 'timed_out': False, 'total': 2506, 'updated': 0, 'created': 2506, 'deleted': 0, 'batches': 3, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []})

## Semantic Search Using KNN without Chatgpt response

In [ ]:
input_keyword = "provide me device for listening music"


response = es.search(
    index="ecommerce_new_index",
    size=5,
    knn={
        "field": "ml.embeddings",
        "k": 10,  # Number of nearest neighbors to return as top hits.
        "num_candidates": 500,  # Number of nearest neighbor candidates to consider per shard. Increasing num_candidates tends to improve the accuracy of the final k results.
        "query_vector_builder": {  # Object indicating how to build a query_vector. kNN search enables you to perform semantic search by using a previously deployed text embedding model.
            "text_embedding": {
                "model_id": ".multilingual-e5-small",  # Text embedding model id "multilingual-e5-small"
                "model_text": input_keyword,  # Query
            }
        },
    },
)

for hit in response["hits"]["hits"]:

    score = hit["_score"]
    product = hit["_source"]["product"]
    category = hit["_source"]["category"]
    description = hit["_source"]["description"]
    print(
        f"\nScore: {score}\nProduct: {product}\nCategory: {category}\nDescription: {description}\n"
    )




Score: 0.930022
Product: JBL Endurance DIVE Waterproof MP3 Player
Category: Sports Equipment
Description: is a waterproof MP3 player with built-in earbuds, perfect for listening to music while swimming or water sports.


Score: 0.9296545
Product: Bose SoundLink Revolve+ Portable Bluetooth Speaker
Category: Bluetooth Speakers
Description: is a portable and water-resistant Bluetooth speaker with 360-degree sound. It's great for indoor and outdoor listening.


Score: 0.9239327
Product: JBL Flip 5 Portable Bluetooth Speaker
Category: Bluetooth Speakers
Description: is a portable and waterproof Bluetooth speaker with powerful sound. It's perfect for outdoor gatherings and music on the go.


Score: 0.92382467
Product: UE Boom 3 Portable Bluetooth Speaker
Category: Bluetooth Speakers
Description: is a rugged and waterproof Bluetooth speaker with 360-degree sound. It's great for outdoor adventures and pool parties.


Score: 0.923139
Product: Yamaha P-515 Portable Digital Piano
Category: Music

## Semantic Search Using KNN with Chatgpt response

In [ ]:
import os
from elasticsearch import Elasticsearch
import openai



OPENAI_API_KEY = 'OPENAI_API_KEY'
openai.api_key = OPENAI_API_KEY

# Initialize the Elasticsearch client
es = Elasticsearch(cloud_id="My_deployment:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyQwYWQ2NTBlMzJiMTE0OGU4YWUyNTUwNmI0NmE3Yjc2MiRkZGIyYTdhZTc3YjY0NzRmOTdhZjAwNjJhNWI1NGM1Ng==", basic_auth=("elastic", "mx8WQ6gXqkAayGL6EfMNrmj5"), request_timeout=3600)

# KNN search parameters
input_keyword = "provide me mens fashion clothes"

# Perform the KNN search
response = es.search(
    index="ecommerce_new_index",
    size=5,
    knn={
        "field": "ml.embeddings",
        "k": 10,
        "num_candidates": 500,
        "query_vector_builder": {
            "text_embedding": {
                "model_id": ".multilingual-e5-small",
                "model_text": input_keyword,
            }
        },
    },
)

# Process the search results
search_result = ''
for hit in response["hits"]["hits"]:
    score = hit["_score"]
    product = hit["_source"]["product"]
    category = hit["_source"]["category"]
    description = hit["_source"]["description"]
    search_result += f"Score: {score}\nProduct: {product}\nCategory: {category}\nDescription: {description}\n\n"

# Generate a response using ChatGPT-3.5
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Answer this user query: " + input_keyword + " with the following context: " + search_result}
    ]
)

# Retrieve and print the response
chatgpt_response = completion.choices[0].message['content']
print(f"Response: {chatgpt_response}")
print(f"Source Information: \n{search_result}")


Response: Certainly! Here are some men's fashion clothing options for a modern and fashionable look:

1. Slim Fit Striped Shirt: A must-have shirt that offers a slim fit and stylish stripes for a modern appearance.
  
2. Tailored Dress Pants: A pair of well-fitted dress pants that exude sophistication and can be easily dressed up or down.

3. Classic Leather Jacket: Elevate your outfit with a timeless leather jacket that adds a rugged yet refined touch to your style.

4. Chelsea Boots: Complete your look with a pair of sleek Chelsea boots that are versatile enough for both casual and formal occasions.

5. Statement Watch: Add a touch of elegance with a statement watch that not only tells time but also serves as a fashion accessory.

These clothing items will help you achieve a fashionable and polished look for various occasions.
Source Information: 
Score: 0.9108627
Product: Closet Rod Dividers
Category: Storage & Organization
Description: are dividers that fit on closet rods, helping 

# Post Process

## Install necessary Packages

In [ ]:
!pip install eland elasticsearch
!pip install elasticsearch
!pip install openai==0.28
!pip install streamlit
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307

## Import necessary modules

In [ ]:
from elasticsearch import (Elasticsearch,helpers)
import getpass
import openai
import os

## Connect to Elasticsearch Cloud

In [ ]:
# Elastic cloud connection
CLOUD_ID = getpass.getpass("Enter Elastic Cloud ID:  ")   

# Password for the 'elastic' user generated by Elasticsearch
ELASTIC_PASSWORD = getpass.getpass("Enter Elastic password:  ")  

# Create the client instance
es = Elasticsearch(cloud_id=CLOUD_ID, basic_auth=("elastic", ELASTIC_PASSWORD), request_timeout=3600)

# Check the connection
try:
    # Ping the Elasticsearch cluster
    if es.ping():
        print("You have Successfully Connected with Elastic Cloud")
    else:
        print("Failed to connect to Elastic Cloud")
except Exception as e:
    print(f"Error connecting to Elastic Cloud: {e}")

Enter Elastic Cloud ID:  ··········
Enter Elastic password:  ··········
You have Successfully Connected with Elastic Cloud


## Semantic Search using KNN without chatgpt response

In [ ]:
from elasticsearch import Elasticsearch


es = Elasticsearch(cloud_id="My_deployment:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyQyYTFmZTY2Yjk5MTQ0YmJjOWE3NjQ1NmNhYWM4ZmEyMSQ5NTY0Zjk5OTI0ODk0YTk1YjI0NDUyYTJiZTJhNmEzNg==", basic_auth=("elastic", "Y0H9aCTnbBrR6LPoQdnx17a5"), request_timeout=3600)



input_keyword = "provide me device for listening music"

response = es.search(
    index="ecommerce_new_index",
    size=5,
    knn={
        "field": "ml.embeddings",
        "k": 10,  # Number of nearest neighbors to return as top hits.
        "num_candidates": 500,  # Number of nearest neighbor candidates to consider per shard. Increasing num_candidates tends to improve the accuracy of the final k results.
        "query_vector_builder": {  # Object indicating how to build a query_vector. kNN search enables you to perform semantic search by using a previously deployed text embedding model.
            "text_embedding": {
                "model_id": ".multilingual-e5-small",  # Text embedding model id ".multilingual-e5-small"
                "model_text": input_keyword,  # Query
            }
        },
    },
)

for hit in response["hits"]["hits"]:

    score = hit["_score"]
    product = hit["_source"]["product"]
    category = hit["_source"]["category"]
    description = hit["_source"]["description"]
    print(
        f"\nScore: {score}\nProduct: {product}\nCategory: {category}\nDescription: {description}\n"
    )




Score: 0.930022
Product: JBL Endurance DIVE Waterproof MP3 Player
Category: Sports Equipment
Description: is a waterproof MP3 player with built-in earbuds, perfect for listening to music while swimming or water sports.


Score: 0.9296545
Product: Bose SoundLink Revolve+ Portable Bluetooth Speaker
Category: Bluetooth Speakers
Description: is a portable and water-resistant Bluetooth speaker with 360-degree sound. It's great for indoor and outdoor listening.


Score: 0.9239327
Product: JBL Flip 5 Portable Bluetooth Speaker
Category: Bluetooth Speakers
Description: is a portable and waterproof Bluetooth speaker with powerful sound. It's perfect for outdoor gatherings and music on the go.


Score: 0.92382467
Product: UE Boom 3 Portable Bluetooth Speaker
Category: Bluetooth Speakers
Description: is a rugged and waterproof Bluetooth speaker with 360-degree sound. It's great for outdoor adventures and pool parties.


Score: 0.923139
Product: Yamaha P-515 Portable Digital Piano
Category: Music

## Semantic Search using KNN with chatgpt response

In [ ]:
# response with chatgpt
from elasticsearch import Elasticsearch
import os
import openais

# Set the API key for OpenAI
os.environ['OPENAI_API_KEY'] = "OPENAI_API_KEY"
openai.api_key = os.environ.get("OPENAI_API_KEY")

# Initialize the Elasticsearch client
es = Elasticsearch(cloud_id="My_deployment:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyQwYWQ2NTBlMzJiMTE0OGU4YWUyNTUwNmI0NmE3Yjc2MiRkZGIyYTdhZTc3YjY0NzRmOTdhZjAwNjJhNWI1NGM1Ng==", basic_auth=("elastic", "Y0H9aCTnbBrR6LPoQdnx17a5"), request_timeout=3600)

# KNN search parameters
input_keyword = "provide me earphones"

# Perform the KNN search
response = es.search(
    index="ecommerce_new_index",
    size=5,
    knn={
        "field": "ml.embeddings",
        "k": 10,
        "num_candidates": 500,
        "query_vector_builder": {
            "text_embedding": {
                "model_id": ".multilingual-e5-small",
                "model_text": input_keyword,
            }
        },
    },
)

# Process the search results
search_result = ''
for hit in response["hits"]["hits"]:
    score = hit["_score"]
    product = hit["_source"]["product"]
    category = hit["_source"]["category"]
    description = hit["_source"]["description"]
    search_result += f"Score: {score}\nProduct: {product}\nCategory: {category}\nDescription: {description}\n\n"

# Generate a response using ChatGPT-3.5
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Answer this user query: " + input_keyword + " with the following context: " + search_result}
    ]
)

# Retrieve and print the response
chatgpt_response = completion.choices[0].message['content']
print(f"Response: {chatgpt_response}")
print(f"Source Information: \n{search_result}")




Response: Based on the context provided, here are some earphone options for you:

1. Jaybird Vista True Wireless Sport Earbuds
- Category: Sports Equipment
- Description: These wireless earbuds are specifically designed for sports and workouts, offering excellent sound quality and a secure fit.

2. Beats Powerbeats Pro Wireless Earphones
- Category: Wireless Earbuds
- Description: These true wireless earphones are sweat and water resistant, providing powerful sound and a secure fit ideal for sports and workouts.

3. Beats Studio Buds Wireless Earphones
- Category: Wireless Earbuds
- Description: These true wireless earphones feature active noise cancellation, sweat and water resistance, immersive sound, and a comfortable fit.

These earphone options cater to different needs and preferences, so you can choose based on your specific requirements for sports, workouts, or everyday use.
Source Information: 
Score: 0.9170399
Product: Jaybird Vista True Wireless Sport Earbuds
Category: Sports

# Streamlit App Code

###  
1. Create one file "app.py" and copy this code and paste in "app.py" file then save it

2. Run "!pip install streamlit -q" code for installing streamlit in colab

3. Run "!wget -q -O - ipv4.icanhazip.com" for getting tunnel port

4. Run "!streamlit run app.py & npx localtunnel --port 8501" we get our link click on link then past tunnel port for redirecting our application

In [ ]:
import os
import streamlit as st
from elasticsearch import Elasticsearch
import openai

# Set the API key for OpenAI
os.environ['OPENAI_API_KEY'] = "OPENAI_API_KEY"
openai.api_key = os.environ.get("OPENAI_API_KEY")

# Initialize the Elasticsearch client
try:
    es = Elasticsearch(cloud_id="My_deployment:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyQwYWQ2NTBlMzJiMTE0OGU4YWUyNTUwNmI0NmE3Yjc2MiRkZGIyYTdhZTc3YjY0NzRmOTdhZjAwNjJhNWI1NGM1Ng==", basic_auth=("elastic", "Y0H9aCTnbBrR6LPoQdnx17a5"), request_timeout=3600)
except ConnectionError as e:
    st.error(f"Connection Error: {e}")

if es.ping():
    st.success("Successfully connected to Elasticsearch!")
else:
    st.error("Oops! Cannot connect to Elasticsearch.")

# KNN search function
def search(input_keyword):
    response = es.search(
        index="ecommerce_new_index",
        size=5,
        knn={
            "field": "ml.embeddings",
            "k": 10,
            "num_candidates": 500,
            "query_vector_builder": {
                "text_embedding": {
                    "model_id": ".multilingual-e5-small",
                    "model_text": input_keyword,
                }
            },
        },
    )
    results = response["hits"]["hits"]
    return results

# Function to generate ChatGPT-3.5 response
def generate_chatgpt_response(input_keyword, search_results):
    search_result_text = ""
    for hit in search_results:
        score = hit["_score"]
        product = hit["_source"]["product"]
        category = hit["_source"]["category"]
        description = hit["_source"]["description"]
        search_result_text += f"Score: {score}\nProduct: {product}\nCategory: {category}\nDescription: {description}\n\n"

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Answer this user query: " + input_keyword + " with the following context: " + search_result_text}
        ]
    )

    chatgpt_response = completion.choices[0]['message']['content']
    return chatgpt_response

# UI using Streamlit
def main():
    st.title("Search Ecommerce Product")

    # Input: User enters search query
    search_query = st.text_area("Enter your search query")

    # Button: User triggers the search
    if st.button('Search'):
        if search_query:
            # Perform the search and get results
            results = search(search_query)

            # Display search results
            st.subheader("Search Results")
            for result in results:
                with st.container():
                    if '_source' in result:
                        try:
                            st.header(f"{result['_source']['product']}")
                        except Exception as e:
                            st.error(f"Error: {e}")

                        try:
                            st.write(f"Score: {result['_score']}")
                        except Exception as e:
                            st.error(f"Error: {e}")

                        st.divider()

                        try:
                            st.write(f"Category: {result['_source']['category']}")
                        except Exception as e:
                            st.error(f"Error: {e}")

                        st.divider()

                        try:
                            st.write(f"Description: {result['_source']['description']}")
                        except Exception as e:
                            st.error(f"Error: {e}")

                        st.divider()

            # Generate and display ChatGPT-3.5 response
            chatgpt_response = generate_chatgpt_response(search_query, results)
            st.subheader("ChatGPT-3.5 Response")
            st.write(chatgpt_response)

if __name__ == "__main__":
    main()


In [ ]:
!pip install streamlit -q

In [ ]:
!wget -q -O - ipv4.icanhazip.com

104.196.209.33


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://104.196.209.33:8501

Need to install the following packages:
  localtunnel@2.0.2
Ok to proceed? (y) y
your url is: https://silly-parents-know.loca.lt
  Stopping...
^C


# Gradio App Code

In [ ]:
import os
from elasticsearch import Elasticsearch
import openai
import gradio as gr

# Set the API key for OpenAI
os.environ['OPENAI_API_KEY'] = "OPENAI_API_KEY"
openai.api_key = os.environ.get("OPENAI_API_KEY")

# Initialize the Elasticsearch client
try:
    es = Elasticsearch(
        cloud_id="My_deployment:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyQwYWQ2NTBlMzJiMTE0OGU4YWUyNTUwNmI0NmE3Yjc2MiRkZGIyYTdhZTc3YjY0NzRmOTdhZjAwNjJhNWI1NGM1Ng==",
        basic_auth=("elastic", "Y0H9aCTnbBrR6LPoQdnx17a5"),
        request_timeout=3600
    )
    if es.ping():
        print("Successfully connected to Elasticsearch!")
    else:
        print("Oops! Cannot connect to Elasticsearch.")
except Exception as e:
    print(f"Connection Error: {e}")

# KNN search function
def search(input_keyword):
    try:
        response = es.search(
            index="ecommerce_new_index",
            size=5,
            knn={
                "field": "ml.embeddings",
                "k": 10,
                "num_candidates": 500,
                "query_vector_builder": {
                    "text_embedding": {
                        "model_id": ".multilingual-e5-small",
                        "model_text": input_keyword,
                    }
                },
            },
        )
        results = response["hits"]["hits"]
        return results
    except Exception as e:
        print(f"Error performing search: {e}")
        return []

# Function to generate ChatGPT-3.5 response
def generate_chatgpt_response(input_keyword, search_results):
    search_result_text = ""
    for hit in search_results:
        score = hit["_score"]
        product = hit["_source"]["product"]
        category = hit["_source"]["category"]
        description = hit["_source"]["description"]
        search_result_text += f"Score: {score}\nProduct: {product}\nCategory: {category}\nDescription: {description}\n\n"

    try:
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": "Answer this user query: " + input_keyword + " with the following context: " + search_result_text}
            ]
        )
        chatgpt_response = completion.choices[0]['message']['content']
        return chatgpt_response
    except Exception as e:
        print(f"Error generating ChatGPT response: {e}")
        return "Error generating response."

# Function to handle the search and generate responses
def handle_search(search_query):
    results = search(search_query)

    # Display search results
    search_result_str = ""
    for result in results:
        if '_source' in result:
            try:
                search_result_str += f"Product: {result['_source']['product']}\n"
            except Exception as e:
                search_result_str += f"Error: {e}\n"

            try:
                search_result_str += f"Score: {result['_score']}\n"
            except Exception as e:
                search_result_str += f"Error: {e}\n"

            search_result_str += "-------------------------\n"

            try:
                search_result_str += f"Category: {result['_source']['category']}\n"
            except Exception as e:
                search_result_str += f"Error: {e}\n"

            search_result_str += "-------------------------\n"

            try:
                search_result_str += f"Description: {result['_source']['description']}\n"
            except Exception as e:
                search_result_str += f"Error: {e}\n"

            search_result_str += "=========================\n"

    chatgpt_response = generate_chatgpt_response(search_query, results)

    return search_result_str, chatgpt_response

# Gradio interface
iface = gr.Interface(
    fn=handle_search,
    inputs=gr.Textbox(lines=2, placeholder="Enter your search query"),
    outputs=[gr.Textbox(label="Search Results"), gr.Textbox(label="ChatGPT-3.5 Response")],
    title="Search Ecommerce Product",
    description="Enter a search query to find products and get responses from ChatGPT-3.5 based on search results."
)

iface.launch(share=True, debug=True)


Successfully connected to Elasticsearch!
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://4a73be6160f99c3da7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://4a73be6160f99c3da7.gradio.live
